In [1]:
import torch

In [2]:
def t(x, y):
    g = torch.tensor(9.81, requires_grad=False)
    n = len(x)
    time = torch.tensor(0., requires_grad=True)
    v_curr = torch.tensor(0., requires_grad=True)
    for i in range(n-1):
        v_curr_new = torch.sqrt(v_curr**2 + 2*g*(y[i]-y[i+1])) 
        time = time + (v_curr_new-v_curr)/g/(y[i]-y[i+1])*torch.sqrt((y[i]-y[i+1])**2+(x[i]-x[i+1])**2)
        v_curr = v_curr_new
        
    return time

### Vanilla Gradient Descent

In [3]:
xstart, xend = 0, 10
ystart, yend = 10, 0

lr = torch.tensor(1, requires_grad=False)
iterations = 1000

x = np.linspace(xstart, xend, 20, endpoint=True)

y_curr = ystart + (x-xstart)*(yend-ystart)/(xend-xstart)

y_curr = torch.tensor(y_curr, requires_grad=True)

y_all = np.zeros((iterations, len(x)))

y_all[0] = y_curr.clone().detach()

times_all = np.zeros(iterations)

for i in range(iterations-1):
    time = t(x, y_curr)
    time.backward()
    grad = y_curr.grad
    grad[0] = torch.tensor(0.)
    grad[-1] = torch.tensor(0.)
    
    y_new = y_curr - lr*grad
    
    y_all[i+1] = y_new.clone().detach()
    times_all[i] = time.clone().detach()
    
    y_curr = torch.tensor(y_all[i+1], requires_grad=True)
    
times_all[-1] = t(x, y_curr).clone().detach()

In [4]:
np.savetxt('Brachistochrone_data/y_all_plain.txt', y_all)

In [5]:
np.savetxt('Brachistochrone_data/times_all_plain.txt', times_all)

### Gradient Descent with Momentum

In [6]:
xstart, xend = 0, 10
ystart, yend = 10, 0

lr = torch.tensor(1, requires_grad=False)
momentum = 0.9
iterations = 1000

x = np.linspace(xstart, xend, 20, endpoint=True)

y_curr = ystart + (x-xstart)*(yend-ystart)/(xend-xstart)

y_curr = torch.tensor(y_curr, requires_grad=True)
velocity = torch.zeros_like(y_curr)

y_all = np.zeros((iterations, len(x)))

y_all[0] = y_curr.clone().detach()

times_all = np.zeros(iterations)

for i in range(iterations-1):
    time = t(x, y_curr)
    time.backward()
    grad = y_curr.grad
    grad[0] = torch.tensor(0.)
    grad[-1] = torch.tensor(0.)
    
    velocity = momentum * velocity + (1 - momentum) * grad
    
    y_new = y_curr - lr*velocity
    
    y_all[i+1] = y_new.clone().detach()
    times_all[i] = time.clone().detach()
    
    y_curr = torch.tensor(y_all[i+1], requires_grad=True)
    
times_all[-1] = t(x, y_curr).clone().detach()

In [7]:
np.savetxt('Brachistochrone_data/y_all_momentum.txt', y_all)

In [8]:
np.savetxt('Brachistochrone_data/times_all_momentum.txt', times_all)